Very simple notebook to test functionality (used in debug mode). Can be turned into short tutorial.


In [ ]:
import numpy as np
import networkx as nx
from graph import Metagraph


In [2]:
#load default networkx graph
g = nx.generators.star_graph(10)
# add betweeness attribute to nodes
bb = nx.betweenness_centrality(g)
nx.set_node_attributes(g, bb, "betweenness")

In [3]:
# add some random edge attributes
att = {(0, 1): "red",
       (0, 2): "blue",
       (0, 5): "red",
       (0, 8): "blue"}
nx.set_edge_attributes(g, att, "color")

In [4]:
#create metagraph class
mg = Metagraph([g], [{"color"}],  [{"betweenness"}])

In [5]:
#attempt dump
mg.dump('./mgraph')

In [6]:
new_graph = Metagraph()
new_graph.load('./mgraph.zip')

In [7]:
adj, natt, eatt = new_graph.get_adjacency()
